In [2]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [3]:
bigM = 1000000

In [4]:
# Load data
locations_df = pd.read_csv('locations.csv')
order_list_df = pd.read_excel('order_list_1.xlsx')
travel_matrix_df = pd.read_csv('travel_matrix.csv')
trucks_df = pd.read_csv('trucks.csv')

In [42]:
Q = sorted(list(set(trucks_df['truck_max_weight'])))
Q[:10]

[2800, 4700, 7000, 9200, 17000]

In [48]:
Q1 = [Q[0]]*5 + [Q[1]]*1 + [Q[2]]*2 + [Q[3]]*7 + [Q[4]]*4

In [49]:
sum(Q1)

165100

In [33]:
max_capacity_w = {v: Q[v] for v in range(len(Q1))}
max_capacity_w

{0: 2800,
 1: 2800,
 2: 2800,
 3: 2800,
 4: 2800,
 5: 2800,
 6: 2800,
 7: 2800,
 8: 2800,
 9: 4700,
 10: 4700,
 11: 4700,
 12: 4700,
 13: 4700,
 14: 4700,
 15: 4700,
 16: 4700,
 17: 7000,
 18: 7000}

In [7]:
dest1 = list(set(order_list_df['Destination Code']))

In [8]:
dest = [str(i) for i in dest1]

In [9]:
order_list_df = order_list_df[order_list_df['Destination Code'].isin(dest1)]

In [34]:
order_list_df1 = order_list_df.sort_values(by = 'Destination Code').groupby('Destination Code').sum("Total Weight").reset_index()
sum(order_list_df1["Total Weight"])

84544.96

In [11]:
locations_df = locations_df[locations_df['location_code'].isin(dest + ['A123'])]

In [12]:
# Convert loading/unloading windows to minutes with explicit format
locations_df['start_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_start'], format='%H:%M').dt.minute
locations_df['end_minutes'] = pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.hour * 60 + pd.to_datetime(locations_df['location_loading_unloading_window_end'], format='%H:%M').dt.minute

In [13]:
customers = locations_df.sort_values(by = 'location_code').iloc[:len(order_list_df1),:]

In [14]:
locations_df2 = locations_df.sort_values(by = 'location_code')

In [15]:
locations_df2

,location_code,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
104,10208117,['3 tonner Box'],8:00,22:00,480,1320
105,10208119,['3 tonner Box'],8:00,22:00,480,1320
156,10208120,['3 tonner Box'],8:00,22:00,480,1320
114,10208499,['3 tonner Box'],8:00,22:00,480,1320
59,10208575,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
...,...,...,...,...,...,...
37,13005196,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
120,13015040,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
115,13067405,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
116,13067657,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320


In [16]:
cap_df = dict(zip(trucks_df['truck_type'], trucks_df['truck_max_weight']))

In [17]:
max_veh_access = []
for i in locations_df2.index:
    max_veh_access.append(cap_df[eval(locations_df2['trucks_allowed'][i])[-1]])

In [18]:
max_veh_access = max_veh_access[len(order_list_df1):] + max_veh_access[:len(order_list_df1)]

In [19]:
max_veh_access

[17000,
 2800,
 2800,
 2800,
 2800,
 7000,
 7000,
 7000,
 7000,
 7000,
 2800,
 2800,
 2800,
 7000,
 2800,
 7000,
 7000,
 2800,
 2800,
 2800,
 7000,
 2800,
 2800,
 7000,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 7000,
 9200,
 2800,
 2800,
 2800,
 2800,
 9200,
 17000,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 7000,
 7000,
 2800,
 7000,
 9200,
 2800,
 2800,
 2800,
 2800,
 7000,
 7000,
 2800,
 2800,
 2800,
 2800,
 2800,
 9200,
 7000,
 2800,
 2800,
 2800,
 2800,
 7000,
 7000,
 2800,
 17000,
 17000,
 7000,
 2800,
 17000,
 2800,
 2800,
 2800,
 7000,
 2800,
 2800,
 7000,
 9200,
 7000,
 2800,
 9200,
 2800,
 2800,
 7000,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 7000,
 7000,
 9200,
 7000,
 17000,
 9200,
 9200,
 9200]

In [20]:
depot = locations_df.sort_values(by = 'location_code').iloc[len(order_list_df1):,:]

In [21]:
Nodes = pd.concat([depot,customers], ignore_index = True)

In [22]:
Nodes

,location_code,trucks_allowed,location_loading_unloading_window_start,location_loading_unloading_window_end,start_minutes,end_minutes
0,A123,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,18:00,480,1080
1,10208117,['3 tonner Box'],8:00,22:00,480,1320
2,10208119,['3 tonner Box'],8:00,22:00,480,1320
3,10208120,['3 tonner Box'],8:00,22:00,480,1320
4,10208499,['3 tonner Box'],8:00,22:00,480,1320
...,...,...,...,...,...,...
104,12995436,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
105,13005196,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
106,13015040,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320
107,13067405,"['3 tonner Box', '5 tonner Box', '7.5 tonner T...",8:00,22:00,480,1320


In [23]:
vehicles = [ k for k in range(0,len(Q))]
customers = [ i for i in range(1,len(Nodes))]
nodes = [ i for i in range(0, len(Nodes))]

In [24]:
sum(Q)

257400

In [25]:
demand = [0] + list(order_list_df1['Total Weight'])

In [26]:
Q

[2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 2800,
 4700,
 4700,
 4700,
 4700,
 4700,
 4700,
 4700,
 4700,
 7000,
 7000,
 7000,
 7000,
 7000,
 7000,
 7000,
 7000,
 7000,
 7000,
 9200,
 9200,
 9200,
 9200,
 17000,
 17000,
 17000,
 17000,
 17000]

In [27]:
start_time = list(Nodes['start_minutes'])
finish_time = list(Nodes['end_minutes'])

In [28]:
# Constants
service_time_customer = 20  
service_time_depot = 60 

In [29]:
dest2 = ['A123'] + sorted(dest)

In [30]:
dest3 = {} 
for i in range(len(dest2)):
    dest3[dest2[i]] = i

In [31]:
travel_matrix_df

,source_location_code,destination_location_code,travel_distance_in_km,travel_time_in_min
0,12854113,12854113,0.00,0
1,12854113,12854149,27.06,29
2,12854113,12577977,15.71,24
3,12854113,10212334,82.55,145
4,12854113,12585577,93.28,147
...,...,...,...,...
76724,A123,12793479,2.38,4
76725,A123,12854131,28.13,33
76726,A123,12996267,23.25,26
76727,A123,13016256,61.55,99


In [32]:
travel_matrix_df = travel_matrix_df[(travel_matrix_df['source_location_code'].isin(dest + ['A123'])) & (travel_matrix_df['destination_location_code'].isin(dest + ['A123']))]

In [33]:
travel_matrix_df['mapped_source'] = travel_matrix_df['source_location_code'].map(dest3)
travel_matrix_df['mapped_destination'] = travel_matrix_df['destination_location_code'].map(dest3)

C:\Users\Acer\AppData\Local\Temp\ipykernel_14376\1226250793.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  travel_matrix_df['mapped_source'] = travel_matrix_df['source_location_code'].map(dest3)
C:\Users\Acer\AppData\Local\Temp\ipykernel_14376\1226250793.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  travel_matrix_df['mapped_destination'] = travel_matrix_df['destination_location_code'].map(dest3)


In [34]:
dist_matrix = {}
time_matrix = {}
for i in travel_matrix_df.index:
    dist_matrix[(travel_matrix_df['mapped_source'][i], travel_matrix_df['mapped_destination'][i])] = travel_matrix_df['travel_distance_in_km'][i]
    time_matrix[(travel_matrix_df['mapped_source'][i], travel_matrix_df['mapped_destination'][i])] = travel_matrix_df['travel_time_in_min'][i]

In [35]:
dist_matrix

{(82, 82): 0.0,
 (82, 94): 27.06,
 (82, 101): 39.79,
 (82, 42): 40.91,
 (82, 76): 13.98,
 (82, 98): 33.74,
 (82, 78): 35.7,
 (82, 83): 17.45,
 (82, 100): 12.43,
 (82, 105): 14.44,
 (82, 80): 13.13,
 (82, 77): 20.98,
 (82, 45): 9.99,
 (82, 19): 40.41,
 (82, 37): 47.56,
 (82, 72): 76.31,
 (82, 65): 46.34,
 (82, 39): 139.23,
 (82, 66): 112.9,
 (82, 53): 4.66,
 (82, 70): 16.2,
 (82, 38): 23.35,
 (82, 57): 40.57,
 (82, 56): 36.07,
 (82, 18): 8.64,
 (82, 62): 6.84,
 (82, 17): 27.45,
 (82, 47): 23.44,
 (82, 64): 23.41,
 (82, 63): 19.72,
 (82, 23): 30.56,
 (82, 5): 30.37,
 (82, 60): 36.93,
 (82, 15): 26.3,
 (82, 8): 30.51,
 (82, 7): 33.82,
 (82, 20): 38.2,
 (82, 35): 14.7,
 (82, 61): 14.55,
 (82, 73): 16.1,
 (82, 6): 22.65,
 (82, 36): 14.19,
 (82, 88): 27.3,
 (82, 69): 14.64,
 (82, 71): 12.77,
 (82, 44): 63.98,
 (82, 49): 12.6,
 (82, 12): 33.81,
 (82, 29): 35.25,
 (82, 34): 33.08,
 (82, 43): 31.73,
 (82, 92): 23.34,
 (82, 84): 35.05,
 (82, 68): 33.31,
 (82, 52): 32.98,
 (82, 104): 30.31,
 (82,

In [36]:
demand

[0,
 314.88,
 376.0,
 48.0,
 219.0,
 168.0,
 232.0,
 283.5,
 170.0,
 1056.0,
 220.0,
 40.0,
 48.0,
 3000.0,
 187.6,
 660.88,
 180.0,
 88.0,
 194.0,
 386.0,
 178.0,
 416.0,
 148.0,
 567.0,
 328.0,
 152.0,
 284.0,
 360.0,
 385.6,
 180.0,
 440.0,
 72.0,
 72.0,
 212.0,
 726.0,
 611.0,
 2299.0,
 1081.0,
 418.0,
 209.0,
 1135.0,
 2200.0,
 1150.0,
 24.0,
 310.0,
 266.0,
 230.0,
 304.0,
 180.0,
 384.0,
 188.0,
 1532.0,
 768.0,
 290.0,
 1254.0,
 2717.0,
 214.5,
 311.0,
 84.0,
 231.0,
 556.0,
 604.0,
 1060.0,
 272.0,
 528.0,
 180.0,
 188.0,
 2090.0,
 510.0,
 836.0,
 417.0,
 320.0,
 191.0,
 1045.0,
 1045.0,
 1500.0,
 1672.0,
 2906.0,
 4800.0,
 180.0,
 1254.0,
 376.0,
 1120.0,
 900.0,
 1200.0,
 600.0,
 480.0,
 760.0,
 1035.0,
 1200.0,
 696.0,
 7200.0,
 400.0,
 672.0,
 1800.0,
 720.0,
 240.0,
 500.0,
 648.0,
 848.0,
 900.0,
 984.0,
 1264.0,
 1728.0,
 960.0,
 2903.0,
 220.0,
 836.0,
 418.0]

In [37]:
nodes

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108]

In [38]:
customers

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108]

In [39]:
dist_matrix

{(82, 82): 0.0,
 (82, 94): 27.06,
 (82, 101): 39.79,
 (82, 42): 40.91,
 (82, 76): 13.98,
 (82, 98): 33.74,
 (82, 78): 35.7,
 (82, 83): 17.45,
 (82, 100): 12.43,
 (82, 105): 14.44,
 (82, 80): 13.13,
 (82, 77): 20.98,
 (82, 45): 9.99,
 (82, 19): 40.41,
 (82, 37): 47.56,
 (82, 72): 76.31,
 (82, 65): 46.34,
 (82, 39): 139.23,
 (82, 66): 112.9,
 (82, 53): 4.66,
 (82, 70): 16.2,
 (82, 38): 23.35,
 (82, 57): 40.57,
 (82, 56): 36.07,
 (82, 18): 8.64,
 (82, 62): 6.84,
 (82, 17): 27.45,
 (82, 47): 23.44,
 (82, 64): 23.41,
 (82, 63): 19.72,
 (82, 23): 30.56,
 (82, 5): 30.37,
 (82, 60): 36.93,
 (82, 15): 26.3,
 (82, 8): 30.51,
 (82, 7): 33.82,
 (82, 20): 38.2,
 (82, 35): 14.7,
 (82, 61): 14.55,
 (82, 73): 16.1,
 (82, 6): 22.65,
 (82, 36): 14.19,
 (82, 88): 27.3,
 (82, 69): 14.64,
 (82, 71): 12.77,
 (82, 44): 63.98,
 (82, 49): 12.6,
 (82, 12): 33.81,
 (82, 29): 35.25,
 (82, 34): 33.08,
 (82, 43): 31.73,
 (82, 92): 23.34,
 (82, 84): 35.05,
 (82, 68): 33.31,
 (82, 52): 32.98,
 (82, 104): 30.31,
 (82,

In [40]:
#Decision Variable

my_model=gp.Model('CVRPTW')

xijk=my_model.addVars(nodes, nodes, vehicles, vtype=GRB.BINARY, name='xijk')
sik = my_model.addVars(nodes, vehicles, vtype = GRB.CONTINUOUS, name = 'sik',lb = 0)
q = my_model.addVars(customers, vehicles, vtype = GRB.CONTINUOUS, name = 'qjk', lb = 0)
I = my_model.addVars(vehicles, vtype = GRB.BINARY, name = 'I')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-24


In [41]:
#Objective function
obj_fn = (gp.quicksum(dist_matrix[i,j]* gp.quicksum(xijk[i,j,k] for k in vehicles) for i in nodes for j in nodes) +
          gp.quicksum(I[k] for k in vehicles)
         )

my_model.setObjective(obj_fn, GRB.MINIMIZE)


In [42]:
#Source to sink constraints
my_model.addConstrs(gp.quicksum(xijk[0,j,k] for j in customers)<=1 for k in vehicles);

In [43]:
my_model.addConstrs(gp.quicksum(xijk[i,0,k] for i in customers)<=1 for k in vehicles);

In [44]:
# my_model.addConstr(gp.quicksum(xijk[0,j,k] for j in customers for k in vehicles)<=10);

In [45]:
my_model.addConstrs(gp.quicksum(xijk[i,j,k] for i in nodes)- gp.quicksum(xijk[j,i,k] for i in nodes)==0 
                    for j in nodes for k in vehicles);

In [46]:
#capacity constraints
my_model.addConstrs(gp.quicksum(q[j,k] for k in vehicles) == demand[j] 
                    for j in customers); 
my_model.addConstrs(gp.quicksum(q[j,k] for j in customers) <= Q[k] for k in vehicles);

In [47]:
my_model.addConstrs(q[j,k] <= bigM * gp.quicksum(xijk[i,j,k] for i in nodes) for j in customers for k in vehicles);

In [48]:
my_model.addConstrs(sik[i,k] + time_matrix[i,j] + service_time_customer
                    - sik[j,k] <= (1-xijk[i,j,k]) *bigM 
                    for i in customers 
                    for j in customers for k in vehicles);

In [49]:
my_model.addConstrs(sik[i,k] <= finish_time[i] for i in nodes for k in vehicles)
                    
my_model.addConstrs(sik[i,k] >= start_time[i] for i in nodes for k in vehicles);                                   

In [50]:
for i in nodes:
    for j in nodes:
        for k in vehicles:
            if Q[k] <= max_veh_access[j] and Q[k] <= max_veh_access[i]:
                my_model.addConstr(xijk[i,j,k] <= 1);
            else:
                my_model.addConstr(xijk[i,j,k] == 0);

In [51]:
my_model.addConstrs(gp.quicksum(xijk[0,j,k] for j in customers) <= I[k] for k in vehicles);

In [52]:
my_model.setParam('MIPGap', 0.6)  
my_model.setParam('MIPFocus',1)
my_model.setParam('Heuristics', 0.5)    

Set parameter MIPGap to value 0.6
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 0.5


In [ ]:
my_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 887516 rows, 447663 columns and 3065080 nonzeros
Model fingerprint: 0x499dc866
Variable types: 8029 continuous, 439634 integer (439634 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [3e-02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 735890 rows and 292446 columns
Presolve time: 1.31s
Presolved: 151626 rows, 155217 columns, 906119 nonzeros
Variable types: 3842 continuous, 151375 integer (151375 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.08s

Barrier performed 0 iterations in 10.33 seconds (3.94 work units)
Barrier solve interru

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\ipykernel\iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


In [ ]:
# Create a new dictionary to store non-zero values
non_zero_xijk = {}

# Iterate over all keys and values in xijk
for (i, j, k), value in xijk.items():
    # Check if the value is a Gurobi variable
    if hasattr(value, "X"):  # If it's a Gurobi variable, use .X to get the optimized value
        if value.X > 0.5:  # Filter for non-zero values
            non_zero_xijk[(i, j, k)] = value.X
    else:  # Otherwise, assume it's a numeric value
        if value > 0.5:  # Filter for non-zero values
            non_zero_xijk[(i, j, k)] = value

# Print the non-zero values
print("Non-zero values of x:")
for (i, j, k), value in non_zero_xijk.items():
    print(f"x_{i}_{j}_{k} = {value}")


Non-zero values of x:
x_0_5_9 = 1.0
x_0_7_3 = 1.0
x_0_10_0 = 1.0
x_0_12_8 = 1.0
x_0_14_18 = 1.0
x_1_0_0 = 1.0
x_2_0_9 = 1.0
x_3_0_3 = 1.0
x_3_0_8 = 1.0
x_4_0_18 = 1.0
x_5_2_9 = 1.0
x_6_12_0 = 1.0
x_7_15_3 = 1.0
x_8_13_18 = 1.0
x_9_3_8 = 1.0
x_10_6_0 = 1.0
x_11_9_8 = 1.0
x_12_1_0 = 1.0
x_12_11_8 = 1.0
x_13_4_18 = 1.0
x_14_8_18 = 1.0
x_15_3_3 = 1.0


In [ ]:
# Create a dictionary to store routes for each vehicle
vehicle_routes = {}

# Iterate over the non_zero_xijk dictionary to create routes
for (i, j, k), value in non_zero_xijk.items():
    if value == 1.0:
        if k not in vehicle_routes:
            vehicle_routes[k] = []  # Create a list for each vehicle if not already created
        vehicle_routes[k].append((i, j))

# Display the routes for each vehicle
for vehicle, route in vehicle_routes.items():
    print(f"Vehicle {vehicle} Route: {route}")

Vehicle 9 Route: [(0, 5), (2, 0), (5, 2)]
Vehicle 3 Route: [(0, 7), (3, 0), (7, 15), (15, 3)]
Vehicle 0 Route: [(0, 10), (1, 0), (6, 12), (10, 6), (12, 1)]
Vehicle 8 Route: [(0, 12), (3, 0), (9, 3), (11, 9), (12, 11)]
Vehicle 18 Route: [(0, 14), (4, 0), (8, 13), (13, 4), (14, 8)]


In [ ]:
max_veh_access

[17000,
 2800,
 2800,
 7000,
 7000,
 2800,
 2800,
 2800,
 7000,
 2800,
 2800,
 7000,
 9200,
 17000,
 9200,
 9200]

In [ ]:
non_zero_q = {(i,k):q[i,k].X for i in customers for k in vehicles if q[i,k].X > 0.1}
non_zero_q

{(1, 0): 385.6,
 (2, 9): 1135.0,
 (3, 3): 1532.0,
 (4, 18): 1254.0,
 (5, 9): 1500.0,
 (6, 0): 180.0,
 (7, 3): 376.0,
 (8, 18): 400.0,
 (9, 8): 240.0,
 (10, 0): 500.0,
 (11, 8): 984.0,
 (12, 0): 1728.0,
 (13, 18): 418.0,
 (14, 18): 220.0,
 (15, 3): 418.0}

In [ ]:
demand

[0,
 385.6,
 1135.0,
 1532.0,
 1254.0,
 1500.0,
 180.0,
 376.0,
 400.0,
 240.0,
 500.0,
 984.0,
 1728.0,
 418.0,
 220.0,
 418.0]

In [ ]:
I

{0: <gurobi.Var I[0] (value 1.0)>,
 1: <gurobi.Var I[1] (value -0.0)>,
 2: <gurobi.Var I[2] (value 0.0)>,
 3: <gurobi.Var I[3] (value 1.0)>,
 4: <gurobi.Var I[4] (value -0.0)>,
 5: <gurobi.Var I[5] (value -0.0)>,
 6: <gurobi.Var I[6] (value -0.0)>,
 7: <gurobi.Var I[7] (value -0.0)>,
 8: <gurobi.Var I[8] (value 1.0)>,
 9: <gurobi.Var I[9] (value 1.0)>,
 10: <gurobi.Var I[10] (value 0.0)>,
 11: <gurobi.Var I[11] (value 0.0)>,
 12: <gurobi.Var I[12] (value 0.0)>,
 13: <gurobi.Var I[13] (value 0.0)>,
 14: <gurobi.Var I[14] (value 0.0)>,
 15: <gurobi.Var I[15] (value 0.0)>,
 16: <gurobi.Var I[16] (value 0.0)>,
 17: <gurobi.Var I[17] (value 0.0)>,
 18: <gurobi.Var I[18] (value 1.0)>,
 19: <gurobi.Var I[19] (value 0.0)>,
 20: <gurobi.Var I[20] (value 0.0)>,
 21: <gurobi.Var I[21] (value 0.0)>,
 22: <gurobi.Var I[22] (value 0.0)>,
 23: <gurobi.Var I[23] (value 0.0)>,
 24: <gurobi.Var I[24] (value 0.0)>,
 25: <gurobi.Var I[25] (value 0.0)>,
 26: <gurobi.Var I[26] (value 0.0)>,
 27: <gurobi.Var

In [ ]:
#use dest3 for reverse mapping into location codes from index